In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
from youtube_transcript_api import YouTubeTranscriptApi

time.sleep(5)

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

## Get URLs for TrainerRoad weekly podcasts

In [2]:
# Go to playlist for weekly podcasts
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.youtube.com/playlist?list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH')
time.sleep(3)

In [3]:
#Scroll to get all podcasts on one page
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
time.sleep(3)

In [4]:
# Get the html
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
# Get the links to each episode
link_divs = soup.find_all(class_="yt-simple-endpoint style-scope ytd-playlist-video-renderer")
links = []
for div in link_divs:
    links.append(div.find('a')['href'])
print(len(links))
print(links[:3])
print(links[-3:])

153
['/watch?v=xXXIACCJ2io&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=1', '/watch?v=iw7AKGGBwnQ&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=2', '/watch?v=g1VFTGRZjLQ&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=3']
['/watch?v=2VkBw9aeb_0&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=151', '/watch?v=Lpq7LpP31Ok&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=152', '/watch?v=eWZVAazaDco&list=PLrKJ0zeMQrI61wqXR8MUBXBHPk5Rli1yH&index=153']


## Get transcripts
Transcripts are in a dictionary.  
Each key in the dictionary is a video id, and the value is the transcript.  
Each transcript is a list of dictionaries, each with keys: text, start, and duration

In [7]:
# Get video ID from each link
video_ids = []
for vid_id in links:
    video_ids.append(vid_id.split('=')[1].split('&')[0])
print(len(video_ids))
print(video_ids[:3])
print(video_ids[-3:])

153
['xXXIACCJ2io', 'iw7AKGGBwnQ', 'g1VFTGRZjLQ']
['2VkBw9aeb_0', 'Lpq7LpP31Ok', 'eWZVAazaDco']


In [11]:
# Get transcripts
transcripts ={}
for vid_id in video_ids:
    try:
        transcripts[vid_id] = YouTubeTranscriptApi.get_transcript(vid_id)
    except:
        pass

In [13]:
# Pickle the transcripts
with open('transcripts.pickle', 'wb') as to_write:
    pickle.dump(transcripts, to_write)

## Some episodes are missing because transcripts are unavil, one is repeated 3x

In [33]:
print(len(video_ids))
print(len(transcripts))

153
141


In [42]:
tkeys = list(transcripts.keys())
import numpy as np
missing_vids = np.setdiff1d(video_ids,tkeys)
print(len(missing_vids))

10


In [45]:
for video in video_ids:
    if video_ids.count(video) > 1:
        print(video)

wu7afTnGAlA
wu7afTnGAlA
wu7afTnGAlA


# Get Date, Likes, Reviews, Title

In [64]:
driver = webdriver.Chrome(chromedriver)

episode_details = {}

for link in links:
    # links is a list of partial urls...apped those to www.youtube.com
    url = 'https://www.youtube.com' + link
    vid_id = link.split('=')[1].split('&')[0]
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Get metrics
    upload_date = soup.find(itemprop="uploadDate")['content']
    
    thumbs = soup.find_all(class_="style-scope ytd-toggle-button-renderer style-text",id="text")
    thumb_list = []
    for thumb in thumbs:
        thumb_list.append(thumb.text)
    likes = thumb_list[0]
    dislikes = thumb_list[1]
    
    views = soup.find(class_="view-count style-scope yt-view-count-renderer").text.split(' ')[0]
    
    title = soup.find(class_="title style-scope ytd-video-primary-info-renderer").find('yt-formatted-string').text
    
    # Put all those metrics in a dictionary
    episode_details[vid_id] = {'likes':likes,\
                              'dislikes':dislikes,\
                              'views':views,\
                              'title': title}


In [66]:
with open('episode_details.pickle', 'wb') as to_write:
    pickle.dump(episode_details, to_write)

In [65]:
len(episode_details)

151